In [ ]:
"""
🧼 Data Cleaning Script for MenuPage.csv

Cleaning Steps:
1. Trimming: Applies TRIM() to all text fields to remove leading/trailing whitespace.
2. NULL & Blank Filtering: Removes rows where 'menu_page_id' or 'menu_id' is NULL or empty.
3. Duplicate Removal: Drops exact duplicate rows after trimming.
4. Export: Saves the cleaned result to 'MenuPage_cleaned.csv'.
"""

In [12]:
#Step 1: Load and Inspect CSV
import pandas as pd
import sqlite3

# Load CSV
df = pd.read_csv('MenuPage.csv')

# Check first few rows
df.head()

,id,menu_id,page_number,image_id,full_height,full_width,uuid
0,119,12460,1.0,1603595,7230.0,5428.0,510d47e4-2955-a3d9-e040-e00a18064a99
1,120,12460,2.0,1603596,5428.0,7230.0,510d47e4-2956-a3d9-e040-e00a18064a99
2,121,12460,3.0,1603597,7230.0,5428.0,510d47e4-2957-a3d9-e040-e00a18064a99
3,122,12460,4.0,1603598,7230.0,5428.0,510d47e4-2958-a3d9-e040-e00a18064a99
4,123,12461,1.0,1603591,7230.0,5428.0,510d47e4-2959-a3d9-e040-e00a18064a99


In [13]:
# Step 2:  Create SQLite DB and Load Table
# Connect to SQLite (creates menu.db if it doesn't exist)
conn = sqlite3.connect('menu.db')

# Store the DataFrame in a table
df.to_sql('MenuPage', conn, if_exists='replace', index=False)


66937

In [16]:
# Step 3: Run SQL query to clean data
# 1. Trimming: Applies TRIM() to all text fields to remove leading/trailing whitespace.
# 2. NULL & Blank Filtering: Removes rows where 'menu_page_id' or 'menu_id' is NULL or empty.
# 3. Duplicate Removal: Drops exact duplicate rows after trimming.
query = """
SELECT 
    id,
    menu_id,
    page_number,

    -- Full height: show as-is and convert NULLs to 'MISSING' for display
    full_height,
    CASE 
        WHEN full_height IS NULL THEN 'MISSING'
        ELSE CAST(full_height AS TEXT)
    END AS full_height_cleaned,

    -- Full width: same as above
    full_width,
    CASE 
        WHEN full_width IS NULL THEN 'MISSING'
        ELSE CAST(full_width AS TEXT)
    END AS full_width_cleaned,

    -- Clean text fields
    image_id,
    TRIM(image_id) AS image_id_cleaned,

    uuid,
    TRIM(uuid) AS uuid_cleaned

FROM MenuPage
WHERE 
    id IS NOT NULL
    AND menu_id IS NOT NULL
    AND page_number IS NOT NULL
"""


cleaned_df = pd.read_sql_query(query, conn)


In [17]:
# Step 4: Drop duplicates
cleaned_df = cleaned_df.drop_duplicates()

# Step 5: Save cleaned result
cleaned_df.to_csv("MenuPage_cleaned.csv", index=False)

# Optional: Show result
cleaned_df.head()

,id,menu_id,page_number,full_height,full_height_cleaned,full_width,full_width_cleaned,image_id,image_id_cleaned,uuid,uuid_cleaned
0,119,12460,1.0,7230.0,7230.0,5428.0,5428.0,1603595,1603595,510d47e4-2955-a3d9-e040-e00a18064a99,510d47e4-2955-a3d9-e040-e00a18064a99
1,120,12460,2.0,5428.0,5428.0,7230.0,7230.0,1603596,1603596,510d47e4-2956-a3d9-e040-e00a18064a99,510d47e4-2956-a3d9-e040-e00a18064a99
2,121,12460,3.0,7230.0,7230.0,5428.0,5428.0,1603597,1603597,510d47e4-2957-a3d9-e040-e00a18064a99,510d47e4-2957-a3d9-e040-e00a18064a99
3,122,12460,4.0,7230.0,7230.0,5428.0,5428.0,1603598,1603598,510d47e4-2958-a3d9-e040-e00a18064a99,510d47e4-2958-a3d9-e040-e00a18064a99
4,123,12461,1.0,7230.0,7230.0,5428.0,5428.0,1603591,1603591,510d47e4-2959-a3d9-e040-e00a18064a99,510d47e4-2959-a3d9-e040-e00a18064a99
